In [ ]:
from keras.optimizers import SGD
from opt_einsum import paths
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from package_from_scratch.CNNs.ResNet import ResNet
from package_from_scratch.CNNs.utils import SimplePreprocessor, ImageToArrayPreprocessor, SimpleDatasetLoader
import matplotlib.pyplot as plt
import numpy as np
print("[INFO] loading images...")
path_to_dataset = "D:/MCUT/Neural Network/datasets/animals"
imagePaths = list(paths.list_images(path_to_dataset))
sp = SimplePreprocessor(32, 32, 3)
iap = ImageToArrayPreprocessor()

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
# convert values to between 0-1
data = data.astype("float") / 255.0

# partition our data into training and test sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25,
                                                  random_state=42)

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# initialize the optimizer and model
print("[INFO] compiling model...")

# initialize stochastic gradient descent with learning rate of 0.005
# how to tune learning rates ?????
opt = SGD(lr=0.005)
# opt = Adam(lr=0.001)
# Instantiate AlexNet architecture
# input image size 32x32
# output class is 3

model = ResNet.build(width=32, height=32, depth=3, classes=3,
                     stages=(9, 9, 9), filters=(64, 64, 128, 256),
                     reg=0.0005,bnEps=2e-5, bnMom=0.9)
model.summary()
# compile the model
# loss function: cross-entropy and optimizer: SGD
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])
no_epochs = 100
no_verbose = 1
no_batch_size = 32  # 32 images will be presented to the network at a time,
# and a full forward and backward pass will be
# done to update the parameters of the network
# train the network
print("[INFO] training network...")

H = model.fit(trainX, trainY, validation_data=(testX, testY),
              batch_size=no_batch_size,
              epochs=no_epochs,
              verbose=no_verbose)

print("[INFO] evaluating network...")

predictions = model.predict(testX, batch_size=no_batch_size)

print(classification_report(
    testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=["cat", "dog", "panda"]
))

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, no_epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, no_epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, no_epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, no_epochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()